In [1]:
import json
from collections import defaultdict

def calculate_credit_score(transactions_data):
    """
    Calculates a credit score for each wallet based on Aave V2 transaction data.
    Scores are an approximation based on net deposits, borrows, and repays.
    Scores are capped between 0 and 1000.
    """
    wallet_metrics = defaultdict(lambda: {
        'net_deposits_usd': 0.0,
        'net_borrows_usd': 0.0,
        'net_repays_usd': 0.0,
    })

    # Process transactions to aggregate metrics per wallet
    for tx in transactions_data:
        # Corrected field names based on your provided JSON sample
        wallet = tx.get('userWallet')
        action = tx.get('action')
        action_data = tx.get('actionData', {}) # Get actionData dictionary, default to empty if not present

        # Extract amount and asset price from actionData
        amount_str = action_data.get('amount')
        asset_price_usd_str = action_data.get('assetPriceUSD')

        if not wallet or not action or amount_str is None or asset_price_usd_str is None:
            # Skip transactions that don't have essential data
            continue

        try:
            # Convert amount and price to float
            # Assuming 'amount' is already in a human-readable format (e.g., 2000 for 2000 USDC)
            amount = float(amount_str)
            asset_price_usd = float(asset_price_usd_str)
            value_usd = amount * asset_price_usd
        except (ValueError, TypeError):
            print(f"Warning: Could not convert amount '{amount_str}' or price '{asset_price_usd_str}' to float. Skipping transaction for wallet {wallet}.")
            continue

        if action == 'deposit':
            wallet_metrics[wallet]['net_deposits_usd'] += value_usd
        elif action == 'borrow':
            wallet_metrics[wallet]['net_borrows_usd'] += value_usd
        elif action == 'repay':
            wallet_metrics[wallet]['net_repays_usd'] += value_usd

    wallet_scores_output = []

    for wallet, metrics in wallet_metrics.items():
        net_deposits = metrics['net_deposits_usd']
        net_borrows = metrics['net_borrows_usd']
        net_repays = metrics['net_repays_usd']

        # Calculate repayment rate
        repayment_rate = 0.0
        if (net_borrows + net_repays) > 0:
            repayment_rate = net_repays / (net_borrows + net_repays)
        else:
            # If no borrows and some repays, assume 100% repayment (e.g., if user only repaid another's loan)
            if net_repays > 0:
                repayment_rate = 1.0

        # Calculate credit score based on simplified logic
        # Base score
        score = 500

        # Reward deposits (scaled)
        score += (net_deposits / 1000)

        # Reward repays (scaled)
        score += (net_repays / 500)

        # Heavily reward good repayment rate
        score += (repayment_rate * 200)

        # Penalize borrows (scaled)
        score -= (net_borrows / 2000)

        # Ensure score is within a reasonable range (e.g., 0 to 1000)
        score = max(0, min(1000, score)) # Cap at 1000 for good performance, 0 for bad

        wallet_scores_output.append({
            'wallet': wallet,
            'net_deposits_usd': round(net_deposits, 2),
            'net_repays_usd': round(net_repays, 2),
            'net_borrows_usd': round(net_borrows, 2),
            'repayment_rate': round(repayment_rate * 100, 2), # as percentage
            'calculated_credit_score': round(score, 2)
        })

    return wallet_scores_output

if __name__ == "__main__":
    json_file_path = 'C:\\Users\\Administrator\\Desktop\\internshala\\user-wallet-transactions.json'

    try:
        with open(json_file_path, 'r') as f:
            transactions = json.load(f)
    except FileNotFoundError:
        print(f"Error: The file '{json_file_path}' was not found. Please ensure it's in the same directory.")
        exit(1)
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{json_file_path}'. Check for malformed JSON.")
        exit(1)

    scores = calculate_credit_score(transactions)

    print("--- Aave V2 Wallet Credit Scores ---")
    # Sort by score in descending order
    sorted_scores = sorted(scores, key=lambda x: x['calculated_credit_score'], reverse=True)
    for entry in sorted_scores:
        print(f"\nWallet: {entry['wallet']}")
        print(f"  Net Deposits: ${entry['net_deposits_usd']:.2f} USD")
        print(f"  Net Repays: ${entry['net_repays_usd']:.2f} USD")
        print(f"  Net Borrows: ${entry['net_borrows_usd']:.2f} USD")
        print(f"  Repayment Rate: {entry['repayment_rate']:.2f}%")
        print(f"  Calculated Credit Score: {entry['calculated_credit_score']:.2f}")

    # You can also save the results to a new JSON file
    # output_file_path = 'aave_wallet_credit_scores.json'
    # with open(output_file_path, 'w') as outfile:
    #     json.dump(sorted_scores, outfile, indent=4)
    # print(f"\nResults saved to '{output_file_path}'")

--- Aave V2 Wallet Credit Scores ---

Wallet: 0x00000000001accfa9cef68cf5371a23025b6d4b6
  Net Deposits: $1987663654.86 USD
  Net Repays: $0.00 USD
  Net Borrows: $0.00 USD
  Repayment Rate: 0.00%
  Calculated Credit Score: 1000.00

Wallet: 0x000000000051d07a4fb3bd10121a343d85818da6
  Net Deposits: $285694480361492676608.00 USD
  Net Repays: $0.00 USD
  Net Borrows: $0.00 USD
  Repayment Rate: 0.00%
  Calculated Credit Score: 1000.00

Wallet: 0x000000000096026fb41fc39f9875d164bd82e2dc
  Net Deposits: $5152310624158186.00 USD
  Net Repays: $0.00 USD
  Net Borrows: $0.00 USD
  Repayment Rate: 0.00%
  Calculated Credit Score: 1000.00

Wallet: 0x0000000002032370b971dabd36d72f3e5a7bf1ee
  Net Deposits: $206577330842852351016960.00 USD
  Net Repays: $70150263790.74 USD
  Net Borrows: $60297620862140793487360.00 USD
  Repayment Rate: 0.00%
  Calculated Credit Score: 1000.00

Wallet: 0x000000000a38444e0a6e37d3b630d7e855a7cb13
  Net Deposits: $18390665686010514374656.00 USD
  Net Repays: $11640